## knock 029 対象データの最頻値を求める

In [49]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列が省略されないようにする

polars.config.Config

### データを読み込む

In [50]:
df_receipt = pl.read_csv("../docker/work/data/receipt.csv")
df_receipt.head()

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001…","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097…","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014…","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000…","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050…","""P060102007""",1,90


### ノック

In [51]:
# P120の店舗コード別の最頻値を求める
(
    df_receipt
    # 店舗コードで集計する
    .group_by(by = "store_cd")
    # 商品コードの最頻値を算出する
    .agg([pl.col("product_cd").mode()])
    .with_columns(
        pl.col("product_cd").list.first().alias("product_cd")
    )
    # 店舗コードで並べ替える
    .sort(by = "store_cd")
    .head(n = 10)
)

store_cd,product_cd
str,str
"""S12007""","""P060303001"""
"""S12013""","""P060303001"""
"""S12014""","""P060303001"""
"""S12029""","""P060303001"""
"""S12030""","""P060303001"""
"""S13001""","""P060303001"""
"""S13002""","""P060303001"""
"""S13003""","""P071401001"""
"""S13004""","""P060303001"""
